In [1]:
"""
This needs to be updated to make sure it works with short ... change all info to be short
"""


# import numpy as np
# from datetime import datetime

# from strat import RSIRisingFalling
# from quantfreedom.enums import CandleBodyType

# from quantfreedom.enums import *
# from quantfreedom.helper_funcs import dl_ex_candles
# from quantfreedom.simulate import run_df_backtest, or_backtest


# np.set_printoptions(formatter={"float_kind": "{:0.2f}".format})

# %load_ext autoreload
# %autoreload 2

In [2]:
candles = dl_ex_candles(
    exchange="mufex",
    symbol="BTCUSDT",
    timeframe="5m",
    since_datetime=datetime(2024, 2, 4),
    until_datetime=datetime(2024, 2, 15),
    # candles_to_dl=3000,
)
timestamps = candles[:, CandleBodyType.Timestamp]

In [3]:
timestamps

array([1707004800000.00, 1707005100000.00, 1707005400000.00, ...,
       1707954300000.00, 1707954600000.00, 1707954900000.00])

In [4]:
long_strat = RSIRisingFalling(
    long_short="long",
    rsi_length=np.array([14]),
    rsi_is_below=np.arange(30, 41, 5),
)
long_strat.set_entries_exits_array(
    candles=candles,
    ind_set_index=2,
)
long_strat.plot_signals(candles=candles)

In [5]:
backtest_settings = BacktestSettings()

exchange_settings = ExchangeSettings(
    asset_tick_step=3,
    leverage_mode=1,
    leverage_tick_step=2,
    limit_fee_pct=0.0003,
    market_fee_pct=0.0006,
    max_asset_size=100.0,
    max_leverage=150.0,
    min_asset_size=0.001,
    min_leverage=1.0,
    mmr_pct=0.004,
    position_mode=3,
    price_tick_step=1,
)

static_os = StaticOrderSettings(
    increase_position_type=IncreasePositionType.RiskPctAccountEntrySize,
    leverage_strategy_type=LeverageStrategyType.Dynamic,
    pg_min_max_sl_bcb="min",
    sl_strategy_type=StopLossStrategyType.SLBasedOnCandleBody,
    sl_to_be_bool=False,
    starting_bar=50,
    starting_available_balance=1000.0,
    static_leverage=None,
    tp_fee_type="limit",
    tp_strategy_type=TakeProfitStrategyType.RiskReward,
    trail_sl_bool=True,
    z_or_e_type=None,
)

dos_arrays = DynamicOrderSettingsArrays(
    max_available_balance_risk_pct=np.array([12]),
    max_trades=np.array([0]),
    account_pct_risk_per_trade=np.array([3]),
    risk_reward=np.array([2, 5]),
    sl_based_on_add_pct=np.array([0.1, 0.5]),
    sl_based_on_lookback=np.array([20, 50]),
    sl_bcb_type=np.array([CandleBodyType.Low]),
    sl_to_be_cb_type=np.array([CandleBodyType.Nothing]),
    sl_to_be_when_pct=np.array([0]),
    trail_sl_bcb_type=np.array([CandleBodyType.Low]),
    trail_sl_by_pct=np.array([1.0]),
    trail_sl_when_pct=np.array([1, 2]),
)

In [6]:
backtest_results = run_df_backtest(
    backtest_settings=backtest_settings,
    candles=candles,
    dos_arrays=dos_arrays,
    exchange_settings=exchange_settings,
    static_os=static_os,
    strategy=long_strat,
)

Starting the backtest now ... and also here are some stats for your backtest.

Total indicator settings to test: 3
Total order settings to test: 16
Total combinations of settings to test: 48
Total candles: 3,168
Total candles to test: 152,064


In [7]:
backtest_results.sort_values(by=["qf_score"], ascending=False).head(10)

,ind_set_idx,dos_index,total_trades,wins,losses,gains_pct,win_rate,qf_score,fees_paid,ending_eq,total_pnl
36,2,4,14.0,8,6,101.176,57.143,0.880,228.894,2011.757,1011.757
37,2,5,14.0,8,6,101.176,57.143,0.880,228.894,2011.757,1011.757
39,2,7,14.0,8,6,93.058,57.143,0.872,217.864,1930.578,930.578
38,2,6,14.0,8,6,93.058,57.143,0.872,217.864,1930.578,930.578
45,2,13,14.0,8,6,172.911,57.143,0.839,246.047,2729.107,1729.107
44,2,12,14.0,8,6,172.911,57.143,0.839,246.047,2729.107,1729.107
47,2,15,14.0,8,6,163.465,57.143,0.833,234.032,2634.646,1634.646
46,2,14,14.0,8,6,163.465,57.143,0.833,234.032,2634.646,1634.646
31,1,15,10.0,5,5,96.473,50.000,0.806,93.063,1964.727,964.727
29,1,13,10.0,5,5,96.445,50.000,0.806,93.649,1964.447,964.447


In [8]:
order_records_df = or_backtest(
    backtest_settings=backtest_settings,
    candles=candles,
    dos_arrays=dos_arrays,
    exchange_settings=exchange_settings,
    static_os=static_os,
    strategy=long_strat,
    ind_set_index=2,
    dos_index=4,
    plot_results=True,
    logger_bool=False,
)

DynamicOrderSettings(
    max_available_balance_risk_pct = 0.12,
    max_trades = 0,
    account_pct_risk_per_trade = 0.03,
    risk_reward = 2.0,
    sl_based_on_add_pct = 0.005,
    sl_based_on_lookback = 20,
    sl_bcb_type = 3,
    sl_to_be_cb_type = 6,
    sl_to_be_when_pct = 0.0,
    trail_sl_bcb_type = 3,
    trail_sl_by_pct = 0.01,
    trail_sl_when_pct = 0.01,
)
IndicatorSettingsArrays(
    rsi_is_above = nan,
    rsi_is_below = 40.0,
    rsi_length = 14,
)


In [9]:
order_records_df

,ind_set_idx,or_set_idx,bar_idx,timestamp,datetime,order_status,available_balance,available_balance,cash_borrowed,cash_used,...,entry_size_usd,entry_price,exit_price,position_size_asset,position_size_usd,realized_pnl,sl_pct,sl_price,tp_pct,tp_price
0,2,4,53,1707020700000,2024-02-04 04:25:00,EntryFilled,1000.000,946.361,4659.755,53.639,...,4713.394,42937.8,NaN,0.110,4713.394,NaN,0.500,42715.9,1.4,43523.2
1,2,4,55,1707021300000,2024-02-04 04:35:00,EntryFilled,1000.000,842.574,13130.559,157.426,...,3914.836,42913.0,NaN,0.201,8628.230,NaN,0.600,42679.4,1.5,43566.0
2,2,4,58,1707022200000,2024-02-04 04:50:00,EntryFilled,1000.000,687.885,25748.085,312.115,...,4301.411,42920.3,NaN,0.301,12929.641,NaN,0.600,42679.4,1.5,43564.1
3,2,4,62,1707023400000,2024-02-04 05:10:00,EntryFilled,1000.000,485.035,41755.655,514.965,...,3592.894,42904.2,NaN,0.385,16522.535,NaN,0.600,42662.2,1.5,43585.8
4,2,4,229,1707073500000,2024-02-04 19:05:00,StopLossFilled,879.629,879.629,NaN,NaN,...,NaN,NaN,42662.2,NaN,NaN,-120.371,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,2,4,2849,1707859500000,2024-02-13 21:25:00,MovedTSL,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.008,49142.3,NaN,NaN
114,2,4,2850,1707859800000,2024-02-13 21:30:00,MovedTSL,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.009,49180.2,NaN,NaN
115,2,4,2879,1707868500000,2024-02-13 23:55:00,MovedTSL,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.009,49188.7,NaN,NaN
116,2,4,2936,1707885600000,2024-02-14 04:40:00,EntryFilled,1629.064,868.361,63827.530,760.703,...,28487.149,49336.1,NaN,0.861,42311.236,NaN,NaN,48974.5,1.0,49464.8
